In [1]:
from collections import defaultdict
import os
import pickle
import random
import requests
import time

from IPython.core.debugger import set_trace
import numpy as np
import pandas as pd
import scipy
import scipy.sparse as sp



In [3]:
with open('../data/all_session_skips_train.pkl','rb') as f:
    skips_train = pickle.load(f)
with open('../data/all_session_skips_test.pkl','rb') as f:
    skips_test = pickle.load(f)
with open('../data/all_session_tracks_train.pkl','rb') as f:
    tracks_train = pickle.load(f)
with open('../data/all_session_tracks_test.pkl','rb') as f:
    tracks_test = pickle.load(f)
track_emb = np.load('../data/track_embedding.npy')

In [6]:
max_train = len(tracks_train)
max_test = 10000
train_data = []
test_fit_data = []
test_pred_data = []

# create train data
for i in range(max_train):
    for j, track in enumerate(tracks_train[i]): 
        train_data.append([i,track])


In [7]:
test_fit_data = []
test_pred_data = []
for i in range(max_test):
    for j, track in enumerate(tracks_test[i]):
        if j<10:
            test_fit_data.append([i,track, skips_test[i][j]])

        else:
            test_pred_data.append([i,track, skips_test[i][j]])

In [8]:
user_num = np.array(train_data)[:,0].max() + 1
item_num = np.array(train_data)[:,1].max() + 1

train_mat = defaultdict(int)
# test_mat = defaultdict(int)
for user, item in train_data:
    train_mat[user, item] = 1
# for user, item in test_data:
#     test_mat[user, item] = 1
train_matrix = sp.dok_matrix((user_num, item_num), dtype=np.float32)
dict.update(train_matrix, train_mat)
# test_matrix = sp.dok_matrix((user_num, item_num), dtype=np.float32)
# dict.update(test_matrix, test_mat)

print(train_matrix.shape)

(3308400, 103909)


In [9]:
print(train_matrix.shape)

(3308400, 103909)


In [49]:
def batch_generator(X, batch_size):
    counter=0
    while 1:        
        print('batch start:', batch_size*counter)
        X_batch = X[batch_size*counter:batch_size*(counter+1),:].todense()
        counter += 1
        yield(np.array(X_batch))
        if batch_size*(counter+1)>np.shape(X)[0]:
            break


In [50]:
import keras
from keras import layers
encoding_dim = 32

inp = keras.Input(shape=(item_num,))
encoded = layers.Dense(encoding_dim, activation='relu')(inp)
outp = layers.Dense(item_num, activation='sigmoid')(encoded)

autoencoder = keras.Model(inp, outp)

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [5]:
X_test = train_matrix[-1000:,:].todense()
for i, X in enumerate(batch_generator(train_matrix, batch_size=2560)):
    print("training batch", i, X.shape)
    autoencoder.fit(X, X,
                    epochs=3,
                    batch_size=2560,
                    shuffle=True,
                    validation_data=(X_test, X_test),
                    verbose=1,
                   )
    del X
    if i%10==0:
        autoencoder.save('autoencoder.h5')

NameError: name 'train_matrix' is not defined

In [ ]:
# del train_matrix

In [15]:
# user mid layer weights as track embedding
autoencoder_mid_weights = autoencoder._layers[1].get_weights()[0]
autoencoder_mid_weights.shape

(103909, 256)

In [16]:
test_fit_data=np.array(test_fit_data)
test_pred_data=np.array(test_pred_data)

In [17]:
test_fit_data[:20,]


array([[     0, 103302,      1],
       [     0,  51333,      1],
       [     0,  72014,      1],
       [     0,  23615,      1],
       [     0,  64985,      1],
       [     0,  53613,      1],
       [     0,  29490,      1],
       [     0,  53535,      1],
       [     0,  21471,      1],
       [     0,  53535,      0],
       [     1,  22430,      0],
       [     1,  22430,      0],
       [     1,  56612,      1],
       [     1,  22430,      0],
       [     1,  56612,      0],
       [     1,  43060,      0],
       [     1,  56612,      1],
       [     1,  22430,      0],
       [     1,  56612,      0],
       [     1,  43060,      0]])

In [18]:
test_pred_data[:20,]

array([[     0,  21471,      1],
       [     0,  68649,      1],
       [     0,  41459,      1],
       [     0,  83707,      1],
       [     0,  67462,      1],
       [     0,  93858,      1],
       [     0,  86293,      1],
       [     0,  74387,      1],
       [     0,  14816,      1],
       [     0, 102419,      1],
       [     1,  14657,      0],
       [     1,   9756,      0],
       [     1,  99171,      0],
       [     1,  35002,      0],
       [     1,  93427,      0],
       [     1,  12278,      0],
       [     1,  94129,      0],
       [     1,  64317,      0],
       [     1,  23134,      0],
       [     1,  42242,      0]])

In [36]:
def aa(pred,act):
    s = 0.0
    t = 0.0
    c = 1.0
    for x, y in zip(pred,act):
        if x != 0 and x != 1:
            raise Exception('Invalid prediction should be 0 or 1')
        if x==y:
            s += 1.0
            t += s / c
        c += 1
    return t/len(act)

def acc(pred,act,k):
    if k==0:
        return 1.0*([pred[0]==act[0]])
    else:
        return 1.0*(pred[:k+1]==act[:k+1])

In [45]:
aa_list = []
acc_list = []

use_feature=True

#iterate by user
test_size=1000    #test_fit_data[:,0].max()
for i in range(test_size):
    act = test_pred_data[test_pred_data[:,0]==i][:, 2]
    pred_track = test_fit_data[test_fit_data[:,0]==i][:, 1]
    #take past 3 rece
    look_back=1
    fit_track = test_fit_data[test_fit_data[:,0]==i][:, 1] [-look_back:]
#     import pdb; pdb.set_trace()
    user_emb = autoencoder_mid_weights[fit_track,].mean(axis=0)
    if use_feature:
        user_emb_feat = track_emb[fit_track,].mean(axis=0)
        user_emb = np.concatenate([user_emb, user_emb_feat], axis=0)

    user_last_stats = 1.0*(np.mean(test_fit_data[test_fit_data[:,0]==i][:, 2][-look_back:])>0.5)
    


    sim=[]
    for j in pred_track:
        trac_emb = autoencoder_mid_weights[j,]
        
        if use_feature:
#             import pdb; pdb.set_trace()
            trac_emb_feat = track_emb[j,]
            trac_emb = np.concatenate([trac_emb, trac_emb_feat], axis=0)
        
        sim.append(np.dot(user_emb, trac_emb).sum())
    
    
    if user_last_stats == 1:
        pred = 1.0*(np.array(sim)>-0.3)
    else:
        pred = 1-1.0*(np.array(sim)>-0.3)
    aa_10 = aa(pred,act)
    aa_list.append(aa_10)
    
    acc_10 = acc(pred,act,10)
    acc_list.append(acc_10)
    
    print(i, aa_10)

0 0.0
1 1.0
2 0.716547619047619
3 0.07222222222222222
4 0.28888888888888886
5 0.12190476190476192
6 0.6777777777777778
7 1.0
8 1.0
9 0.7
10 0.5282142857142857
11 0.7
12 0.8
13 0.4041666666666666
14 0.36666666666666664
15 0.6557142857142857
16 0.025
17 0.6
18 0.17619047619047618
19 0.9
20 0.5
21 0.5975396825396825
22 0.0
23 0.07261904761904761
24 0.9
25 1.0
26 0.9
27 0.2873015873015873
28 0.7
29 0.6565476190476189
30 1.0
31 1.0
32 0.49071428571428566
33 0.7
34 0.6484920634920635
35 0.7577777777777778
36 0.9
37 0.6565476190476189
38 0.2814285714285714
39 0.8521031746031745
40 0.6458730158730158
41 0.5808730158730159
42 0.8521031746031745
43 1.0
44 0.07166666666666666
45 0.4084126984126984
46 0.5599999999999999
47 0.9
48 0.2
49 0.7577777777777778
50 1.0
51 0.3071428571428571
52 0.4
53 1.0
54 1.0
55 0.07261904761904761
56 0.5908730158730158
57 0.5684920634920634
58 0.6842063492063492
59 0.8521031746031745
60 0.7
61 0.02
62 0.34174603174603174
63 0.5208333333333334
64 0.0
65 0.0
66 0.0
67 0

In [46]:
np.mean(aa_list)

0.5563255555555555

In [47]:
for k in range(1,11):
    print('acc@%d: '%k, np.mean(np.array(acc_list)[:, :k]))


acc@1:  0.768
acc@2:  0.7295
acc@3:  0.702
acc@4:  0.69025
acc@5:  0.6856
acc@6:  0.6761666666666667
acc@7:  0.6678571428571428
acc@8:  0.6615
acc@9:  0.6542222222222223
acc@10:  0.6454
